In [ ]:
%pip install databricks-sql-connector


In [ ]:
! pip install urllib3==1.26.6

In [ ]:
from databricks import sql
import os
import json

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

DBWS_HOST = os.getenv("dbws_host_domain")
DBWS_HTTP_PATH = os.getenv("dbws_host_path")
DBWS_PAT = os.getenv("dbws_pat")

def execute_sql_st(query):
    result_rows = []

    with sql.connect(server_hostname = DBWS_HOST,
                    http_path        = DBWS_HTTP_PATH,
                    access_token     = DBWS_PAT) as conn:

        with conn.cursor() as cursor:
            cursor.execute(query)
            result = cursor.fetchall()

            for row in result:
                result_rows.append(row)

    if len(result_rows) == 1:
        return result_rows[0]
    
    return result_rows


# sql_query = "SELECT `date` as filter_date FROM hive_metastore.unum_prod2_bi_gold.filter_date_range LIMIT 1"
# sql_query = "SELECT agent_type FROM hive_metastore.unum_prod2_gold.events_askskan where agent_type=0 LIMIT 1"
# sql_query = "SELECT agent_type FROM unum_askskan.events_delta_tb where agent_type=0 LIMIT 1" 
# sql_query = "SELECT * FROM unum_askskan.events_delta_tb  LIMIT 1" 
# sql_query = "SELECT COUNT(*) AS count FROM hive_metastore.unum_askskan.events WHERE event_date >= '2023-04-01' AND event_date <= '2023-04-30'"
sql_query = "DESCRIBE unum_askskan.events_delta_tb"
res= execute_sql_st(sql_query)
print( res  )
print(json.dumps(res, indent=2))

In [ ]:
# to do: verify if the schema in the view matches the schema in the pickle file (just the name).
# Take 5 questions that can be surely answered.

import pandas as pd

def get_column_difference(list_columns, csv_path):
    # Extract the column names from the list of rows
    list_column_names = [row.col_name for row in list_columns]
    
    # Read the CSV file
    df = pd.read_csv(csv_path)
    
    # Get the column names from the CSV
    csv_columns = sorted(df['Column Name'].tolist()) # df.columns.tolist()
    print("CSV cols:", csv_columns)
    
    # Find the difference between the list and CSV column names
    # difference = list(set(list_column_names) - set(csv_columns))
    difference = list(set(csv_columns) - set(list_column_names))
    
    return difference, csv_columns

# Example usage
list_columns = sorted(res)
csv_path = '../askskan/data/original/schema/schema1.csv'

column_difference, csv_columns = get_column_difference(list_columns, csv_path)
print("list cols:", list_columns)
print(json.dumps(list_columns, indent=2))

print("Column difference:", column_difference)

print(json.dumps(column_difference, indent=2))
    

In [ ]:
# Create a DataFrame from the lists
df_csv = pd.DataFrame({'CSV cols': csv_columns})
df_list = pd.DataFrame({'List cols': [row.col_name for row in list_columns]})

# Write the DataFrame to an Excel file
with pd.ExcelWriter('columns_diff.xlsx') as writer:
    df_csv.to_excel(writer, sheet_name='CSV Columns', index=False)
    df_list.to_excel(writer, sheet_name='List Columns', index=False)